In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer
from finrl.finrl_meta.preprocessor.preprocessors import data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
# from finrl.finrl_meta.env_stock_trading.env_stocktrading import EnvSetup
# from finrl.env.EnvMultipleStock_train import StockEnvTrain
# from finrl.env.EnvMultipleStock_trade import StockEnvTrade
# from finrl.model.models import DRLAgent
# from finrl.trade.backtest import BackTestStats, BaselineStats, BackTestPlot, backtest_strat, baseline_strat
# from finrl.trade.backtest import backtest_strat, baseline_strat

import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [2]:

# Download and save the data in a pandas DataFrame:
df = YahooDownloader(start_date = '2008-01-01',
                     end_date = '2020-12-01',
                     ticker_list = config.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
# Perform Feature Engineering:
fe = FeatureEngineer(use_technical_indicator=True,use_turbulence=False)
df = fe.preprocess_data(df.copy())


# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
df.head()

Successfully added technical indicators


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list
0,2008-12-30,3.122143,3.144643,3.025714,2.638855,967601600,AAPL,1,-0.081465,3.123077,2.502889,42.673740,-80.476419,16.129793,2.780282,2.897204,"[[0.0013494475995607929, 0.0004283419557516316..."
1,2008-12-30,57.000000,57.660000,56.820000,44.533276,4300800,AMGN,1,0.159785,45.912545,43.034153,50.840500,36.439602,17.245628,43.780100,43.302341,"[[0.0013494475995607929, 0.0004283419557516316..."
2,2008-12-30,17.820000,18.129999,17.700001,14.507502,11777300,AXP,1,-1.016864,19.177955,13.102037,41.557589,-101.744461,33.966523,16.165652,18.218194,"[[0.0013494475995607929, 0.0004283419557516316..."
3,2008-12-30,40.080002,41.340000,39.810001,30.940775,4549700,BA,1,-0.433255,31.945967,28.948484,45.444026,35.959234,7.693500,30.289959,32.492893,"[[0.0013494475995607929, 0.0004283419557516316..."
4,2008-12-30,42.570000,43.750000,42.009998,30.079878,5060400,CAT,1,0.593094,31.465553,26.078213,50.051060,72.391304,19.456481,27.535549,27.511480,"[[0.0013494475995607929, 0.0004283419557516316..."


In [ ]:

# Initialize env:
env_setup = StockPortfolioEnv(stock_dim = stock_dimension,
                        state_space = state_space,
                        initial_amount = 1000000,
                        tech_indicator_list = config.TECHNICAL_INDICATORS_LIST)
                        
env_train = env_setup.create_env_training(data = train,
                                          env_class = StockPortfolioEnv)  

TypeError: 'numpy.int64' object is not callable